In [8]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import requests

from os import listdir
import unicodedata

from paths import *
from scripts.manipulation import remove_acentos
from scripts.manipulation import normalize_cols
from scripts import scrap_data
from scripts import manipulation
from scripts import io
from scripts import vis_graphs

from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import yaml

import geopandas as gpd

# Taubate

- Nao temos acesso aos dados por municipio dos seguintes indicadores: 
 - Ocupação leitos UTI COVID
 - Leitos COVID/100 mil hab
 - Internações

In [9]:
sp_full = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
sp_full['nome_drs'] = np.where(sp_full['nome_munic']=='São Paulo','Município de São Paulo',sp_full['nome_drs'])




In [10]:
cols = ['datahora','nome_munic','codigo_ibge','casos','casos_novos','obitos','obitos_novos','nome_drs']

mask = sp_full['nome_drs'].isin(['Taubaté'])
vale = sp_full[mask][cols]


rename_cols = {
    "casos":"casos_tbt",
    "casos_novos":"casos_novos_tbt",
    "obitos":"obitos_tbt",
    "obitos_novos":"obitos_novos_tbt",
}
vale_ = vale.groupby(by=['nome_drs','datahora'], as_index=False).sum()

vale_['nome_munic'] = 'Vale'
vale_['codigo_ibge'] = 0


vale = pd.concat([vale,vale_], axis=0)

vale_ = vale_.rename(columns = rename_cols).drop(columns=['nome_munic','codigo_ibge'])
vale = vale.merge(vale_, on=['nome_drs','datahora'])

In [11]:
cols = ['casos','obitos','casos_tbt','obitos_tbt']


dd_final_all = pd.DataFrame()

for municipio in vale['nome_munic'].unique():
    tbt = vale[vale['nome_munic']==municipio]

    for col in cols:
        

        tbt[f'{col}_shift_7'] = tbt[f'{col}'].shift(7)
        tbt[f'{col}_shift_14'] = tbt[f'{col}'].shift(14)


        tbt[f'{col}_7d'] = tbt[f'{col}'] - tbt[f'{col}_shift_7']
        tbt[f'{col}_14d'] = tbt[f'{col}'] - tbt[f'{col}_shift_14']


        tbt[f'{col}_shift_14'] = tbt[f'{col}_7d'].shift(7)

        tbt[f'{col}_var'] = tbt[f'{col}_7d']/tbt[f'{col}_shift_14']
        
        tbt = tbt.drop(columns = [f'{col}_shift_14',f'{col}_shift_7',])



#     dd = tbt[['datahora','nome_munic',f'{col}',f'{col}_7',f'{col}_14d',f'{col}_var']]

    dd_final_all = pd.concat([dd_final_all,tbt], axis=0)


    


vale_final = dd_final_all.copy()

In [12]:
vale_final['casos_tbt'] = 100 * vale_final['casos'] / vale_final['casos_tbt'] 
vale_final['casos_novos_tbt'] = 100 * vale_final['casos_novos'] / vale_final['casos_novos_tbt'] 
vale_final['obitos_tbt'] = 100 * vale_final['obitos'] / vale_final['obitos_tbt'] 
vale_final['obitos_novos_tbt'] = 100 * vale_final['obitos_novos'] / vale_final['obitos_novos_tbt'] 
vale_final = vale_final.fillna(0)

In [13]:
vale_final = vale_final.sort_values(by=['datahora','nome_munic'])
vale_final['codigo_ibge'] = vale_final['codigo_ibge'].astype(int)
vale_final['datahora'] = pd.to_datetime(vale_final['datahora'])

In [14]:
vale_final.tail()

,datahora,nome_munic,codigo_ibge,casos,casos_novos,obitos,obitos_novos,nome_drs,casos_tbt,casos_novos_tbt,obitos_tbt,obitos_novos_tbt,casos_7d,casos_14d,casos_var,obitos_7d,obitos_14d,obitos_var,casos_tbt_7d,casos_tbt_14d,casos_tbt_var,obitos_tbt_7d,obitos_tbt_14d,obitos_tbt_var
12994,2021-01-14,São Sebastião,3550704,2588,15,55,2,Taubaté,3.153790,1.171875,3.190255,9.52381,50.0,80.0,1.666667,8.0,8.0,inf,7468.0,11707.0,1.761736,79.0,139.0,1.316667
12996,2021-01-14,Taubaté,3554102,9559,94,216,0,Taubaté,11.648794,7.343750,12.529002,0.00000,791.0,1195.0,1.957921,1.0,13.0,0.083333,7468.0,11707.0,1.761736,79.0,139.0,1.316667
12997,2021-01-14,Tremembé,3554805,710,3,4,0,Taubaté,0.865221,0.234375,0.232019,0.00000,55.0,79.0,2.291667,0.0,0.0,0.000000,7468.0,11707.0,1.761736,79.0,139.0,1.316667
12998,2021-01-14,Ubatuba,3555406,2817,24,51,2,Taubaté,3.432854,1.875000,2.958237,9.52381,106.0,142.0,2.944444,4.0,4.0,inf,7468.0,11707.0,1.761736,79.0,139.0,1.316667
12999,2021-01-14,Vale,0,82060,1280,1724,21,Taubaté,100.000000,100.000000,100.000000,100.00000,7468.0,11707.0,1.761736,79.0,139.0,1.316667,7468.0,11707.0,1.761736,79.0,139.0,1.316667


### RT

In [15]:
#load br cities
geo_sp = gpd.read_file('../../brasil_geodata/maps/sp_municipios.json')
# cols = ['geocodigo','nome_mesorregiao','geometry']
cols = ['geocodigo','nome_mesorregiao','nome_municipio']
geo_sp = geo_sp[cols]

In [16]:
#load state data
url  = 'http://datasource.coronacidades.org/'
data = 'br/states/rt'

ds = pd.read_csv(f'{url}{data}')

ds['last_updated'] = pd.to_datetime(ds['last_updated'])

sp = ds[ds['state_num_id']==35]
sp = sp.rename(columns={'state_num_id':'city_id'})
sp['city_id'] = 'SP'
sp['nome_mesorregiao'] = 'SP'
sp['nome_municipio']   = 'Estado de SP'

# rj = ds[ds['state']=='RJ']
# rj = rj.rename(columns={'state':'city_id'})
# rj['nome_mesorregiao'] = 'RJ'
# rj['nome_municipio']   = 'Estado do RJ'

# mg = ds[ds['state']=='MG']
# mg = mg.rename(columns={'state':'city_id'})
# mg['nome_mesorregiao'] = 'MG'
# mg['nome_municipio']   = 'Estado de MG'

# am = ds[ds['state']=='AM']
# am = am.rename(columns={'state':'city_id'})
# am['nome_mesorregiao'] = 'AM'
# am['nome_municipio']   = 'Estado de AM'

In [17]:
#load cities data
data = 'br/cities/rt'

df = pd.read_csv(f'{url}{data}')

df['last_updated'] = pd.to_datetime(df['last_updated'])


In [18]:
dd = df.merge(geo_sp, how='right', left_on = 'city_id', right_on = 'geocodigo')
dd.head()


regiao = 'Vale do Paraíba Paulista'
mask = (dd['nome_mesorregiao']==regiao)
vale = dd[mask]
vale = vale[vale['city_id'].notnull()]

mask = (dd['nome_municipio']=='São Paulo')
sp_city = dd[mask]


vale = pd.concat([vale,sp_city], 0)
vale = pd.concat([vale,sp], 0)

vale['geocodigo'] = vale['geocodigo'].fillna(111).astype(int)

In [19]:
vale = vale.rename(columns={'geocodigo':'codigo_ibge','last_updated':'datahora'})
vale['datahora'] = pd.to_datetime(vale['datahora'])

In [20]:
mask = vale['datahora']== max(vale['datahora'])
vale_rt_last = vale[mask]
vale_rt_last = vale_rt_last.drop(columns=['datahora'])
vale_rt_last = vale_rt_last.rename(columns={'nome_municipio':'municipio'})

In [21]:
mask = vale_final['nome_munic']=='Vale'
just_vale = vale_final[mask]
just_vale = just_vale.drop(columns=['nome_munic'])
just_vale['last_update'] = max(just_vale['datahora'])

In [22]:
#upload to drive
from scripts import io
io.to_gbq(vale_final,'vale_covid','covid','gabinete-sv', if_exists='replace')
io.to_gbq(just_vale,'just_vale_covid','covid','gabinete-sv', if_exists='replace')
io.to_gbq(vale,'rt_vale_covid','covid','gabinete-sv', if_exists='replace')
io.to_gbq(vale_rt_last,'vale_rt_last','covid','gabinete-sv', if_exists='replace')
io.to_gbq(just_vale,'just_vale_covid','covid','gabinete-sv', if_exists='replace')

1it [00:09,  9.54s/it]
1it [00:04,  4.34s/it]
1it [00:08,  8.28s/it]
1it [00:04,  4.92s/it]
1it [00:05,  5.34s/it]


## Plano SP

## Casos

In [23]:
from scripts import manipulation_sp

casos_full = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
casos, sp_casos = manipulation_sp.padronize_casos(casos_full)

## Internacoes

In [24]:
internacoes_full = pd.read_csv('https://github.com/seade-R/dados-covid-sp/raw/master/data/plano_sp_leitos_internacoes.csv', sep=';')
internacoes, sp_internacoes = manipulation_sp.padronize_internacoes(internacoes_full)

## Ocupacao UTI

In [25]:
uti = pd.read_csv('../data/sp_gov/uti_ocupacao.csv', sep=';')

uti_final = uti.copy()


# mask =  uti_final['datahora'] == max(uti_final['datahora'])

# uti_final = uti_final[mask]

# uti_final['datahora'] = max(df_final['datahora'])

## Add Fases

In [26]:
#merge all tables
df = manipulation_sp.padronize_planosp(sp_casos,sp_internacoes, uti_final)

# calculate fases
df = manipulation_sp.add_fases(df)

# padronize names and column names
# df = manipulation_sp.padronize_planosp_names(df)

In [27]:
df.head()

,datahora,nome_drs,casos,casos_novos,casos_var,obitos,obitos_novos,obitos_var,obitos_14d,casos_14d,leitos_pc,internacoes_var,internacoes_14d,internacoes_7d,pop,uti_var,internacoes_14d_pc,obitos_14d_pc,Capacidade Hospitalar,Evolução da Pandemia,Classif. Final
0,2020-07-09,Araraquara,3118.0,77.0,0.98,50.0,0.0,1.5,15.0,1088.0,9.74,1.45,419.0,248.0,991435,31.2,42.3,1.5,4.0,2.4,2.4
1,2020-07-16,Araraquara,3764.0,125.0,1.2,67.0,2.0,1.89,26.0,1185.0,9.68,1.0,495.0,247.0,991435,35.4,49.9,2.6,4.0,2.8,2.8
2,2020-07-23,Araraquara,4452.0,176.0,1.07,78.0,3.0,0.65,28.0,1334.0,10.10,0.85,458.0,211.0,991435,44.8,46.2,2.8,4.0,3.2,3.2
3,2020-07-30,Araraquara,5623.0,249.0,1.7,94.0,3.0,1.45,27.0,1859.0,10.72,1.22,469.0,258.0,991435,46.5,47.3,2.7,4.0,2.2,2.2
4,2020-08-06,Araraquara,6761.0,284.0,0.97,105.0,3.0,0.69,27.0,2309.0,15.17,1.08,537.0,279.0,991435,34.9,54.2,2.7,4.0,2.8,2.8


## Add Fases

## Padronize 

In [28]:

cols = ['datahora','nome_drs','Capacidade Hospitalar','uti_var','leitos_pc','Evolução da Pandemia','casos_var','internacoes_var','internacoes_14d_pc','obitos_var','obitos_14d_pc','Classif. Final']

cols = ['datahora','nome_drs','Capacidade Hospitalar','uti_var','leitos_pc','Evolução da Pandemia','casos_var','internacoes_var','obitos_var','Classif. Final']



rename_drs = {
    'Araraquara':'DRS 03 - Araraquara',
    'Araçatuba':'DRS 02 - Araçatuba',
    'Baixada Santista':'DRS 04 - Baixada Santista',
    'Barretos':'DRS 05 - Barretos',
    'Bauru':'DRS 06 - Bauru',
    'Campinas':'DRS 07 - Campinas',
    'Estado de São Paulo':'Estado de São Paulo',
    'Franca':'DRS 08 - Franca',
    'Marília':'DRS 09 - Marília',
    'Município de São Paulo':'DRS 01 - Município de São Paulo',
    'Piracicaba':'DRS 10 - Piracicaba',
    'Presidente Prudente':'DRS 11 - Presidente Prudente',
    'Registro':'DRS 12 - Registro',
    'Ribeirão Preto':'DRS 13 - Ribeirão Preto',
    'Sorocaba':'DRS 16 - Sorocaba',
    'São José do Rio Preto':'DRS 15 - São José do Rio Preto',
    'São João da Boa Vista':'DRS 14 - São João da Boa Vista',
    'Taubaté':'DRS 17 - Taubaté',
    'Estado de São Paulo':'00 - Estado de São Paulo'
}

df['nome_drs'] = df['nome_drs'].map(rename_drs)

ddf = df.copy()


In [29]:
cols_rename = {
    "Capacidade Hospitalar": "cap_hosp",
    "Evolução da Pandemia": "evolucao_pand",
    "Classif. Final": "classif_final",
    "nome_drs": "drs_name",
    'Data':'datahora'
}


ddf = df.rename(columns=cols_rename)
mask = ddf['datahora']==max(ddf['datahora'])
ddf['last_update'] = max(ddf['datahora'])
ddf_last = ddf[mask].rename(columns={"drs_name":"DRS"})



In [30]:
#upload to drive
from scripts import io
io.to_gbq(ddf,'plano_sp','covid','gabinete-sv', if_exists='replace')
io.to_gbq(ddf_last,'plano_sp_last','covid','gabinete-sv', if_exists='replace')

1it [00:04,  4.15s/it]
1it [00:05,  5.09s/it]
